# Demo of optimisation-based reconstruction in CIL

### The case is a 3D parallel-beam synchrotron dataset of a steel wire.

First we import everything we need:

In [ ]:
# Import all CIL framework components needed
from cil.framework import ImageData, ImageGeometry, AcquisitionData, AcquisitionGeometry, BlockDataContainer

# Import algorithms, operators and functions from CIL optimisation module
from cil.optimisation.algorithms import CGLS, GD, FISTA, PDHG
from cil.optimisation.operators import BlockOperator, GradientOperator, IdentityOperator, \
                                       GradientOperator, FiniteDifferenceOperator
from cil.optimisation.functions import IndicatorBox, MixedL21Norm, L2NormSquared, \
                                       BlockFunction, L1Norm, LeastSquares, \
                                       OperatorCompositionFunction, TotalVariation, \
                                       ZeroFunction

# Import CIL Processors for preprocessing
from cil.processors import CentreOfRotationCorrector, Slicer, TransmissionAbsorptionConverter

# Import CIL display function
from cil.utilities.display import show2D

# Import from CIL ASTRA plugin
from cil.plugins.astra.processors import FBP, AstraBackProjector3D
from cil.plugins.astra.operators import ProjectionOperator

# All external imports
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import scipy

Exactly as in the notebook 03_preprocessing from week 1, we load the steel-wire demonstration data provided as part of CIL, carry out some preprocessing and FBP reconstructions for reference:

In [ ]:
# Load the example data set
from cil.utilities.dataexample import SYNCHROTRON_PARALLEL_BEAM_DATA
data_sync = SYNCHROTRON_PARALLEL_BEAM_DATA.get()

# Preprocessing
scale = data_sync.get_slice(vertical=20).mean()
data_sync = data_sync/scale
data_sync = TransmissionAbsorptionConverter()(data_sync)
data_sync = CentreOfRotationCorrector.xcorrelation(slice_index='centre')(data_sync)

# Crop data and reorder for ASTRA backend
data90 = Slicer(roi={'angle':(0,90), 
                     'horizontal':(20,140,1)})(data_sync)
data90.reorder(order='astra')

# Set up and run FBP for 90-angle dataset
ag90 = data90.geometry
ig = ag90.get_ImageGeometry()
recon90 = FBP(ig, ag90, device='gpu')(data90)

# Set up and run FBP for 15-angle dataset
data15 = Slicer(roi={'angle': (0,90,6)})(data90)
ag = data15.geometry
recon15 = FBP(ig, ag, device='gpu')(data15)

In [ ]:
# Define custom display function to reuse for visualizing all reconstructions consistently
sx = 44
sz = 103
ca1 = -0.01
ca2 =  0.11
myshow = lambda vol : show2D(vol, 
                             slice_list=[('horizontal_x',sx),('vertical',sz)], 
                             cmap='inferno', 
                             fix_range=(ca1,ca2), 
                             origin='upper-left')

Show a slices of the 90-degree FBP reconstruction:

In [ ]:
myshow(recon90)

And the 15-projection FBP reconstruction, which contains severe streak artifacts:

In [ ]:
myshow(recon15)

Optimisation-based reconstruction is based on a fully discretized model that is conventionally assumed to be linear:
\\[Au = b \\]
where \\(A\\)  is the linear operator known as the system matrix representing forward projection of an image to its sinogram, \\(b\\) is the sinogram data, and \\(u\\) is the unknown image to be reconstructed.

The first thing we are going to need is the LinearOperator representing forward and back-projections. We set up the ProjectionOperator from the CIL-ASTRA plugin by passing the 15-projection image and acquisition geometries:

In [ ]:
A = ProjectionOperator(ig, ag)

We choose to work with the 15-projection dataset here and refer to it by `b` for convenience:

In [ ]:
b = data15

Unfortunately, as we normally have noise, model errors and other inconsistencies in the data, we cannot expect a solution exists to \\(Au = b\\).  We therefore relax the problem and aim to find a solution that is as close as possible to fitting the data. This is conventionally measured in a least-squares sense in that we solve the leastproblem
\\[ \min_u \|Au - b \|_2^2 \\]
where
\\[\|y \|_2^2 = \sum_i y_i^2\\].
The function that is to be minimized is called the objective function.

The reconstruction is the image \\(u\\) solves the optimisation problem, i.e., that results in the lowest possible value of the objective function, in this case of the (squared) residual norm \\(\|Au - b \|_2^2 \\). In order to find the solution we use an iterative optimisation algorithm. Many exist, the perhaps most basic one is the gradient descent method, which is available in CIL as the GD algorithm. To set it up we need to specify the objective function in terms of a CIL Function. For least-squares this is simply:

In [ ]:
f1 = LeastSquares(A, b)

In iterative algorithms we must provide an initial point from which to start, here we choose the zero image. 

In [ ]:
x0 = ig.allocate(0.0)

`f1` is a CIL Function and CIL Functions can be evaluated for particular input images, for example we can evaluate it (which is the starting objective value of the optimisation problem)  for `x0`:

In [ ]:
f1(x0)

CIL Functions provide a number of methods that are used by optimisation algorithms, most notably, if a function is smooth (continuously differentiable), then a CIL Function provides the `gradient` method using which the gradient of the function can be evaluated at a particular input image. For example we can evaluate the gradient at `x0` and since it contains an element for each voxel, we can display it as an image:

In [ ]:
show2D(f1.gradient(x0))

To set up the gradient descent algorithm, we specify the initial point, the objective function, whether to use a fixed step size or a back-tracking line search (None), the maximal number of iterations to run, and how often to evaluate the objective function value:

In [ ]:
myGD_LS = GD(x_init=x0, 
             objective_function=f1, 
             step_size=None, 
             max_iteration=1000, 
             update_objective_interval=10)

Once the algorithm is set up, we can run it for a specified number of iterations and here using `verbose=1` to print out progress information:

In [ ]:
myGD_LS.run(300, verbose=1)

Once done, the latest image/solution in the algorithm can be shown as an image:

In [ ]:
myshow(myGD_LS.solution)

This was a basic least-squares example. We can specify more involved optimisation problems by combining multiple CIL Functions through addition, scalar multiplication as well as composition with CIL operators. For example, as an alternative to using CGLS to solve the Tikhonov problem with gradient operator D, i.e.,  
\\[\min_u \|Au-b\|_2^2 + \alpha^2\|Du\|_2^2\\]
as covered by a different exercise, we can set this objective function up step by step.

First, we set again the least-squares data fitting term as before:

In [ ]:
f1 = LeastSquares(A, b)

Next we specify the operator `D` in the regularisation term and the value of the regularisation parameter `alpha`:

In [ ]:
D = GradientOperator(ig)
alpha = 10.0

We can construct the regularisation term by composing the squares L2-norm with the operator D as

In [ ]:
f2 = OperatorCompositionFunction(L2NormSquared(),D)

Finally we form the full optimisation problem from the composnents defined:

In [ ]:
f = f1 + (alpha**2)*f2

As before we can set up a gradient descent algorithm to solve this problem:

In [ ]:
myGD_tikh = GD(x_init=x0, 
               objective_function=f, 
               step_size=None, 
               max_iteration=1000, 
               update_objective_interval = 10)

In [ ]:
myGD_tikh.run(200, verbose=1)

In [ ]:
myshow(myGD_tikh.solution)

As an exercise you may specify and solve the same Tikhonov problem using CGLS as in a previous notebook, and compare the result and performance with using the GD algorithm.

Many useful reconstruction methods involve minimisation of functions that are NOT smooth and in those cases we need dedicated optimisation algorithms for non-smooth problems. In this notebook we consider optimisation problems that can be written in the form
\\[\min_u f(u) + g(u)  \\]
where \\(f\\) is a smooth function as before, but \\(g\\) may now be non-smooth. \\(g\\) further needs to be "simple", in a certain sense, namely it should have a proximal mapping that is easy to evaluate. Proximal mapping can be thought of a generalisation of the gradient for a non-smooth function.

For this problem class the algorithm FISTA (Fast iterative shrinkage thresholding algorithm) can be employed. It is also known as the accelerated proximal gradient method.

We consider a couple of examples for different functions \\(g\\). First we consider again least-squares but with a non-negativity constraint on all pixels. This problem can be written 
\\[\min_u \|Au-b\|_2^2 + I_{C}(u) \\]
where $I_{\mathbb{R}^+}(u)$ is a special convex function known as an indicator function, which takes on the value 0 in its convex domain C (which we here take to be the set of images with only nonnegative pixel values), and the (extended) value of \\(\infty\\) outside its domain. This can be specified in CIL using an IndicatorBox function:

In [ ]:
F = LeastSquares(A, b)
G = IndicatorBox(lower=0.0)

A FISTA algorithm instance can be set up similarly to the GD algorithm but specifying the \\(f\\) and \\(g\\) functions separately:

In [ ]:
myFISTANN = FISTA(f=F, 
                  g=G, 
                  x_init=x0, 
                  max_iteration=1000)

We run it and display the resulting solution:

In [ ]:
myFISTANN.run(300, verbose=1)

In [ ]:
myshow(myFISTANN.solution)

We see that the nonnegativity constraint, as expected, prevents any values from being negative. Furthermore, this has a positive effect of removing some of the streak artifacts in the background.

Another possibility is sparsity regularization which we can achieve by choosing \\(g\\) as the L1-norm multiplied by a regularisation parameter \\(\alpha\\) to balance the strength of fitting to the data and enforcing regularisation:
\\[g(u) = \alpha\|u\|_1 = \alpha\sum_u |u_i| \\]

In [ ]:
a1 = 30
G = a1*L1Norm()

As before we set up a FISTA algorithm instance and run it:

In [ ]:
myFISTAL1 = FISTA(f=F, 
                  g=G, 
                  x_init=x0, 
                  max_iteration=1000, 
                  update_objective_interval=10)

In [ ]:
myFISTAL1.run(300,verbose=1)

We show the solution of L1 regularised least-squares produced by FISTA:

In [ ]:
myshow(myFISTAL1.solution)

Here, all small values of the background, and part of the sample, have been forced to be zero by the sparsity regularisation term, keeping only the pixel values of the largest magnitude. Sparsity regularisation does not directly enforce smoothing, which is seen in the image by neighbouring pixel values being rather different in the part of the image that is not zero.

An option that is sometimes better is to enforce sparsity of the image gradient. This is known as Total Variation (TV) regularisation and tends to enforce piecewise constant areas separated by sharp edges. Recall that for example Tikhonov regularisation may reduce noise but tends to blur edges, so TV may have an advantage if the image to be reconstructed is known to consist of relatively homogenuous areas separated by sharp edges. In CIL, TV is available as the TotalVariation function, and we can set up and solve the TV-regularised problem in the same way as before:

In [ ]:
GTV = 0.02*TotalVariation()
myFISTATV = FISTA(f=F, 
                  g=GTV, 
                  x_init=x0 ,
                  max_iteration=1000)

Note that the proximal mapping of TotalVariation is not simple but needs to be evaluated numerically, but this is handled by the TotalVariation function, however it does take a while to run:

In [ ]:
myFISTATV.run(200,verbose=1)

Show the slices of the TV reconstruction by FISTA:

In [ ]:
myshow(myFISTATV.solution)

We that TV-regularisation succesfully compensates for the streak artifacts caused by few projections, suppresses noise and preserves sharp edges in the reconstruction.

An even more flexible algorithm for non-smooth problems is the Primal Dual Hybrid Gradient (PDHG) algorithm, which also goes under other names such as the Chambolle-Pock algorithm. In PDHG we can split complicated functionals into simpler parts for which the proximal mapping can be evaluated. PDHG will be covered in more detail in a separate notebook, here it is demonstrated how to set up the same TV-regularized problem we just solved with FISTA. Note how BlockFunctions and BlockOperators are used to specify multiple terms/operators:

In [ ]:
alpha = 0.02
F = BlockFunction(L2NormSquared(b=b), 
                  alpha*MixedL21Norm())
K = BlockOperator(A, 
                  D)
G = ZeroFunction()
myPDHG = PDHG(f=F, 
              g=G, 
              operator=K, 
              max_iteration=10000)

Run the algorithm for a specified number of iterations with increased verbosity/amount of printing to screen:

In [ ]:
myPDHG.run(5000,verbose=2)

Show the TV-regularized solution obtained by the PDHG Algorithm and note it is identical with the one from FISTA:

In [ ]:
myshow(myPDHG.solution)

CIL Algorithms can record history of objective values (primal and dual for PDHG) for monitoring convergence:

In [ ]:
plt.figure()
plt.loglog(myFISTATV.iterations[1:], myFISTATV.objective[1:])
plt.loglog(myPDHG.iterations[1:], myPDHG.objective[1:])
plt.loglog(myPDHG.iterations[1:], myPDHG.dual_objective[1:])
plt.loglog(myPDHG.iterations[1:], np.array(myPDHG.objective[1:])-np.array(myPDHG.dual_objective[1:]))
plt.ylim((1e0,1e5))
plt.legend(['FISTA','PDHG primal','PDHG dual','PDHG gap'])
plt.grid()
plt.xlabel('Number of iterations')
plt.ylabel('Objective value')

We observe that the two algorithms achieve apprximately the same (primal) objective value. PDHG in addition supplies the dual objective, and the primal-dual gap (difference of primal of dual objectives) which helps for monitoring convergence, as it tends to zero as the algorithm approaches the solution.